In [ ]:
PROMPT = """
Given the following cases and their corresponding case ids 


"""

In [1]:
import pandas as pd

df = pd.read_csv('data/echr_qa_dataset.csv')
df

,question,answer,guide,paragraphs,prompt_id
0,How does the Court establish its jurisdiction ...,The first corresponds to the Court’s own juris...,guide_art_1_eng,"[2, 3, 4, 37, 7, 12, 55]",legal-sentence-level-cot-with-search-v2
1,Based on the travaux préparatoires and the mea...,In accordance with Article 31 § 1 of the Vienn...,guide_art_1_eng,"[2, 4, 5, 6, 13, 19, 87]",legal-sentence-level-cot-with-search-v2
2,"Based on the concept of ""jurisdiction"" in publ...",For that purpose it needs to establish its jur...,guide_art_1_eng,"[2, 6, 7, 8, 40, 9, 12, 87]",legal-sentence-level-cot-with-search-v2
3,In the case of alleged violations by a State a...,"In addition, the Court may examine of its own ...",guide_art_1_eng,"[8, 9, 10, 13, 17, 18]",legal-sentence-level-cot-with-search-v2
4,What criteria does the Court use to determine ...,For that purpose it needs to establish its jur...,guide_art_1_eng,"[2, 9, 10, 11, 12, 13, 62]",legal-sentence-level-cot-with-search-v2
...,...,...,...,...,...
616,"In cases involving sexual abuse, how can the r...",In assessing whether the accused received a fa...,guide_art_6_criminal_eng,"[453, 506, 536, 537, 538]",legal-sentence-level-cot-with-search-v1
617,In cases involving criminal proceedings for se..., Witnesses in sexual abuse cases Criminal pr...,guide_art_6_criminal_eng,"[532, 536, 537, 538, 539, 540]",legal-sentence-level-cot-with-search-v2
618,In a case where an accused was only given the ...,Whether an applicant’s direct confrontation wi...,guide_art_6_criminal_eng,"[504, 523, 181, 440, 540, 541, 542]",legal-sentence-level-cot-with-search-v2
619,"In a criminal trial, if a trial court grants a...","However, when a trial court grants a request t...",guide_art_6_criminal_eng,"[544, 545, 546, 547, 296, 297, 507]",legal-sentence-level-cot-with-search-v2


In [2]:
import spacy

nlp = spacy.load("en_core_web_trf")

def get_sentences_spacy(text: str):
    doc = nlp(text)
    return [sentence.text for sentence in doc.sents]

In [3]:
possible_citations_df = pd.read_csv('data/echr_case_law_guides_with_possible_eng_citations.csv')
possible_citations_df

,paragraph,guide_id,paragraph_id,possible_citations,possible_eng_citations
0,"As provided by Article 1, the engagement under...",guide_art_1_eng,1,"{""001-114082"": ""CASE OF CATAN AND OTHERS v. TH...","{""001-114082"": ""CASE OF CATAN AND OTHERS v. TH..."
1,"In the Convention context, the term jurisdicti...",guide_art_1_eng,2,"{""001-114082"": ""CASE OF CATAN AND OTHERS v. TH...","{""001-114082"": ""CASE OF CATAN AND OTHERS v. TH..."
2,"Historically, the text drawn up by the Committ...",guide_art_1_eng,3,"{""001-114082"": ""CASE OF CATAN AND OTHERS v. TH...","{""001-114082"": ""CASE OF CATAN AND OTHERS v. TH..."
3,The adoption of Article 1 of the Convention wa...,guide_art_1_eng,4,"{""001-22099"": ""BANKOVI\u0106 AND OTHERS v. BEL...","{""001-22099"": ""BANKOVI\u0106 AND OTHERS v. BEL..."
4,The travaux préparatoires go on to note that t...,guide_art_1_eng,5,"{""001-22099"": ""BANKOVI\u0106 AND OTHERS v. BEL...","{""001-22099"": ""BANKOVI\u0106 AND OTHERS v. BEL..."
...,...,...,...,...,...
6790,"As regards family reunion, see the thematic Ca...",guide_terrorism_eng,128,"{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED...","{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED..."
6791,"Under Article 15 of the Convention, in the eve...",guide_terrorism_eng,129,"{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED...","{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED..."
6792,It transpires from the Court’s case-law that t...,guide_terrorism_eng,130,"{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED...","{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED..."
6793,The Court had ruled that terrorism in Northern...,guide_terrorism_eng,131,"{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED...","{""001-105612"": ""CASE OF AL-JEDDA v. THE UNITED..."


In [7]:
import json


def get_possible_citations(guide_id: str, paragraphs: list[int]):
    matches = possible_citations_df[(possible_citations_df['guide_id'] == guide_id) & (possible_citations_df['paragraph_id'].isin(paragraphs))]
    possible_citations = {}
    for _, row in matches.iterrows():
        possible_citations_row = row['possible_eng_citations']
        possible_citations_row = json.loads(possible_citations_row)
        for k, v in possible_citations_row.items():
            possible_citations[k] = v
    return possible_citations


print(get_possible_citations('guide_art_1_eng', [1, 2, 3, 4, 5]))

{'001-114082': 'CASE OF CATAN AND OTHERS v. THE REPUBLIC OF MOLDOVA AND RUSSIA', '001-206897': 'SLOVENIA v. CROATIA', '001-222654': 'CASE OF SVETOVA AND OTHERS v. RUSSIA', '001-222889': 'UKRAINE AND THE NETHERLANDS v. RUSSIA', '001-22099': 'BANKOVIĆ AND OTHERS v. BELGIUM AND OTHERS', '001-207622': 'UKRAINE v. RUSSIA (RE CRIMEA)', '001-141152': 'CASE OF STEPHENS v. MALTA  (No. 1)', '001-186258': 'CASE OF VASILICIUC AGAINST THE REPUBLIC OF MOLDOVA', '001-208054': 'CASE OF VERONICA CIOBANU v. THE REPUBLIC OF MOLDOVA', '001-57920': 'CASE OF LOIZIDOU v. TURKEY (PRELIMINARY OBJECTIONS)', '001-58007': 'CASE OF LOIZIDOU v. TURKEY', '001-61875': 'CASE OF ASSANIDZE v. GEORGIA', '001-49227': 'CYPRUS AGAINST TURKEY'}


In [8]:
for _, row in df.iterrows():
    guide_id = row['guide']
    paragraphs = row['paragraphs']
    paragraphs = [int(p) for p in paragraphs.removeprefix('[').removesuffix(']').split(',')]
    print(get_possible_citations(guide_id, paragraphs))
    break

KeyError: 'guide_id'